In [1]:
#from HARK.utilities import CRRAutility, CRRAutilityP
import matplotlib.pyplot as plt
import numpy as np

#CRRAutilityP_hack = lambda c, gam: float('inf') if c == 0.0 else CRRAutilityP(c, gam)

In [2]:
## Doing this because of the CRRAutility warnings

import warnings
warnings.filterwarnings('ignore')

## Consumption with Independent Shocks

_Notebook by Sebastian Benthall_

In this notebook, we will use both HARK and BARK to build and solve a household consumption model with independent shocks, and compare the results.

### With HARK

In [3]:
from HARK.ConsumptionSaving.ConsIndShockModel import IndShockConsumerType
from HARK.ConsumptionSaving.ConsIndShockModel import init_lifecycle
from copy import copy

In [4]:
init_lifecycle

{'cycles': 1,
 'CRRA': 2.0,
 'Rfree': 1.03,
 'DiscFac': 0.96,
 'LivPrb': [0.998566,
  0.998583,
  0.998599,
  0.998609,
  0.998611,
  0.99861,
  0.998601,
  0.998569,
  0.998508,
  0.998419,
  0.998312,
  0.998192,
  0.998056,
  0.997906,
  0.99774,
  0.997556,
  0.997348,
  0.997115,
  0.996852,
  0.996562,
  0.996249,
  0.995916,
  0.995561,
  0.995186,
  0.99479,
  0.994349,
  0.993881,
  0.993428,
  0.993005,
  0.992583,
  0.992124,
  0.991583,
  0.990942,
  0.990175,
  0.98929,
  0.988296,
  0.987216,
  0.986059,
  0.984831,
  0.983509,
  0.982022,
  0.980368,
  0.978602,
  0.976732,
  0.974708,
  0.97243,
  0.969863,
  0.967036,
  0.963933,
  0.960506,
  0.956589,
  0.952211,
  0.947534,
  0.942585,
  0.937209,
  0.931163,
  0.924276,
  0.916534,
  0.907855,
  0.898197,
  0.887532,
  0.8758360000000001,
  0.863084,
  0.849246,
  0.834296],
 'PermGroFac': [1.0419577244168425,
  1.042054094671763,
  1.0419741368106594,
  1.0417252594892363,
  1.0413149489133513,
  1.040750757064985

In [7]:
# Solve the agent over a standard life-cycle

LifeCycleType = IndShockConsumerType(init_lifecycle)

LifeCycleType.cycles = 1  ## life cycle problem instead of infinite horizon
LifeCycleType.vFuncBool = False  ## no need to calculate the value for the purpose here


TypeError: unsupported operand type(s) for -: 'int' and 'dict'

In [ ]:
## solving the model
t0 = time()
LifeCycleType.solve()
LifeCycleType.cFunc = [
    LifeCycleType.solution[t].cFuncAdj for t in range(LifeCycleType.T_cycle)
]
LifeCycleType.ShareFunc = [
    LifeCycleType.solution[t].ShareFuncAdj for t in range(LifeCycleType.T_cycle)
]
t1 = time()
print(
    "Solving a "
    + str(LifeCycleType.T_cycle)
    + " period portfolio choice problem takes "
    + str(t1 - t0)
    + " seconds."
)

In [ ]:
## compare the consumption function for the T-1 period
which_period = 1
mGrid = mGridPort_life[which_period]
plt.plot(mGrid, LifeCycleType.cFunc[which_period - 1](mGrid), "r--", label="HARK")
plt.plot(mGrid, cFuncPort_life[which_period](mGrid), "k-", label="MicroDSOP")
plt.legend(loc=0)
plt.title("consumption function solved by MicroDSOP and HARK")
plt.xlabel("m")
plt.ylabel(r"$c_{T-1}(m)$")

### With BARK

We have predefined some stage definitions in this module:

In [ ]:
import cons_stages

CRRA = cons_stages.CRRA
epsilon = cons_stages.epsilon

In [ ]:
def display_stage(stage):
    print(f"x: {stage.inputs}, k: {stage.shocks}, a: {stage.actions}, y: {stage.outputs}")

The labor stage has no actions. In this stage, the agent experiences exogenous shocks and grows their resources.

In [ ]:
l_stage = cons_stages.labor_stage

display_stage(l_stage)

In [ ]:
def l_v_y_der_terminal(y):
    return CRRAutilityP_hack(y['m'], CRRA)

l_sol = l_stage.solve(
    x_grid = {'b' : np.linspace(epsilon, 10,50)},
    v_y_der = l_v_y_der_terminal, ## Is this working, using the derivative value function here?
    ## Why?
    ### Is there a default set somewhere?
    shock_approx_params = {
            'psi' : 5, 
            'theta' : 5, 
        },
    policy_finder_method = 'opt'
)


$$v'_x(x) = \mathbb{E}_k\left[\frac{\partial q}{\partial x}(x, k, \pi^*(x, k)) \right] = \mathbb{E}_k\left[\beta v'_y(T(x, k, \pi^*(x, k)))\frac{\partial T}{\partial x} (x, k, \pi^*(x, k) \right]$$

$$v'_x(b) = \mathbb{E}_{\psi, \theta} \left[(G\psi)^{\rho - 1} v'_y \left( \frac{b + \theta}{G \psi} \right) \frac{1}{G\psi} \right]$$

In [ ]:
l_sol.dataset

In [ ]:
l_sol.dataset['v_x_der']

In [ ]:
x = np.linspace(0.5, 5,100)

plt.plot(x, [CRRAutilityP_hack(xi, CRRA) for xi in x], label = "l_v_der_terminal")
plt.plot(l_sol.dataset['b'][5:], l_sol.dataset['v_x_der'][5:], label = 'l_sol')
plt.legend()

We can repeat these stages again and again to simulate the household earning and consuming over time.

But there's one small problem: the output of the consumption stage is $m$ while the input of the labor stage is $b$.

We need to transform $m$ to $b$.

In [ ]:
from HARK.stage import Stage, backwards_induction

### TWIST STAGE

def twist(mapper):
    return Stage(
        transition = lambda x, k, a : {mapper[xi] : x[xi] for xi in mapper}, 
        transition_der_x = 1,
        inputs = list(mapper.keys()), 
        actions = [],
        outputs = list(mapper.values())
    )
    
      
twist_stage = twist({'a' : 'b'})



In [ ]:
t_sol = twist_stage.solve(
    x_grid = {'a' : np.linspace(epsilon, 10,50)},
    next_sol = l_sol,
    shock_approx_params = {},
    policy_finder_method = 'opt'
)


In [ ]:
t_sol.dataset

Note this is identical to the marginal value function for the labor stage, but relabeled.

The consumption stage models the transition $a = m - c$ where $c$ is the choice of consumption. It has no shocks.

In [ ]:
c_stage = cons_stages.consumption_stage

c_sol = c_stage.solve(
    y_grid = {'a' : np.linspace(epsilon, 10,50)},
    v_y_der = t_sol.v_x_der,
    policy_finder_method = 'egm'
)

c_sol.dataset

**Revise the order**:

- Start with taking expectations over (labor) shocks
- Consumption stage
- twist, tick as necessary.


**Scoping parameters**

You can share parameters within a tick,
but not across ticks.

Parameters may be shared between stages.

In [ ]:
x_space =  np.linspace(epsilon,4,500)

stages_data = [
    {
        'stage' : c_stage,
        'y_grid' : {'a' : x_space},
        'optimizer_args' :{
            'a0f' : lambda x: x['m'] - epsilon
        },
        'method' : 'egm'
    },
    {
        'stage' : twist({'a' : 'b'}),
        'x_grid' : {'a' : x_space},
    },
    {
        'stage' : l_stage,
        'x_grid' : {
            'b' : x_space,
        },
        'shock_approx_params' : {
            'psi' : 5, 
            'theta' : 5, 
        },
    },
    #tick
]

In [ ]:
sols = backwards_induction(stages_data * 3, x_space, terminal_v_y_der = l_v_y_der_terminal)

In [ ]:
sols[-1].dataset['v_x_der']

In [ ]:
l_sol = l_stage.solve(
    x_grid = {'b' : x_space},
    y_grid = {'m' : x_space},
    v_y_der = l_v_y_der_terminal, ## Is this working, using the derivative value function here?
    ## Why?
    ### Is there a default set somewhere?
    shock_approx_params = {
            'psi' : 5, 
            'theta' : 5, 
        },
    policy_finder_method = 'opt'
)

Note: There is a small difference between the computed $v^{l'}_x$ with backwards induction and the same function computed by passing in the terminal $v^{l'}_y$ directly. This is because in the former case the terminal marginal value function is first passed to an interpolator. The disparity is small, but worse for lower values. 

In [ ]:
plt.plot(sols[-1].dataset['v_x_der'][13:], label="bi")
plt.plot(l_sol.dataset['v_x_der'][13:], label='l_sol')
plt.legend()

In [ ]:
c_sol = c_stage.solve(
    y_grid = {'a' : np.linspace(epsilon, 4,500)},
    next_sol = sols[-2],
    policy_finder_method = 'egm'
)

In [ ]:
plt.plot(c_sol.dataset['v_x_der'][13:])
plt.plot(sols[-3].dataset['v_x_der'][13:])



In [ ]:
plt.plot(sols[-3].dataset['pi*'][3:])
plt.plot(c_sol.dataset['pi*'][3:])

In [ ]:
import matplotlib.pyplot as plt

for i, s in enumerate([sols[i] for i in (range(0,9, 3))]):
    plt.plot(x_space[:50], [s.pi_star({'m' : m,}, {})['c'] for m in x_space[:50]], label = f"pi*_{i}")
    
plt.legend()

In [ ]:
sols[-1].dataset['v_x_der'][13:]

In [ ]:
l_sol.dataset['v_x_der'][:13]

In [ ]:
plt.plot(x_space, [np.log(CRRAutilityP_hack(xi, CRRA)) for xi in x_space])
plt.plot(x_space, np.log(sols[-1].dataset['v_x_der']), label="bi")
plt.legend()

In [ ]:
sols[-1].dataset['v_x_der']